### ETF-holdings recomendations for Accenture, based on predictions from LightFM Model

In [1]:
#Library Installation
#pip install scikit_optimize

In [2]:
# importinhg libraries
import pandas as pd
import os
from scipy.sparse import csr_matrix
import numpy as np
from IPython.display import display_html
import warnings
import time


import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')

from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
from skopt import forest_minimize

def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace(
        'table', 'table style="display:inline"'), raw=True)


# update the working directory to the root of the project
os.chdir('..')
warnings.filterwarnings("ignore")


C:\Users\hites\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
#Data Extraction
base_path1 = "C:\\Users\\hites\\Downloads\\"
etf_df = pd.read_csv(base_path1 + 'data_top15.csv')
etf_df.head()

,ETFSym,HoldingSym,HoldingPercentage
0,HDG,ARGD,0.0001
1,HDG,OSTK,0.0001
2,HDG,FRTA,0.0001
3,HDG,ANGO,0.0001
4,HDG,RGR,0.0001


#### Creation of Matrix

In [4]:
# Pivoting table in order to get a Matrix from the dataframe
etf_interaction = pd.pivot_table(etf_df, index='ETFSym', columns='HoldingSym', values='HoldingPercentage')
# Displaying the Dataframe
etf_interaction

HoldingSym,A,AA,AAAU,AAL,AALB,AAM.PRA,AAN,AAON,AAP,AAPL,...,ZNGA,ZNTL,ZOM,ZS,ZTS,ZUMZ,ZUO,ZURN,ZWS,ZYNE
ETFSym,,,,,,,,,,,,,,,,,,,,,
ACES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACSI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACVF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACWI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YLDE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YOLO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YYY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# fill missing values with 0
etf_interaction = etf_interaction.fillna(0)
etf_interaction

HoldingSym,A,AA,AAAU,AAL,AALB,AAM.PRA,AAN,AAON,AAP,AAPL,...,ZNGA,ZNTL,ZOM,ZS,ZTS,ZUMZ,ZUO,ZURN,ZWS,ZYNE
ETFSym,,,,,,,,,,,,,,,,,,,,,
ACES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACIO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0697,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACSI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACVF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACWI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YLDE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0454,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YOLO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YYY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Dictionary Building for ETF holdings

In [6]:
#Building item disctionary for ETF Holdings
itemdict={}
Stocklist = etf_interaction.columns 

for idx, item in enumerate(Stocklist):

    itemdict[item] = idx


In [7]:
etf_interaction.index

Index(['ACES', 'ACIO', 'ACSI', 'ACVF', 'ACWI', 'ADFI', 'ADME', 'AESR', 'AFLG',
       'AFMC',
       ...
       'XVOL', 'XVV', 'XWEB', 'XYLD', 'XYLG', 'YLDE', 'YOLO', 'YYY', 'ZECP',
       'ZIG'],
      dtype='object', name='ETFSym', length=1046)

In [8]:
id_to_symbol = {}

for key, value in itemdict.items():
    id_to_symbol[value] = key
    
print(id_to_symbol[1])

AA


#### Dictionary Building for ETFs

In [9]:
#Building item disctionary for ETF's 
user_id = list(etf_interaction.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1

#### CSR Matrix Conversion

In [10]:
# convert to csr matrix
etf_interaction_csr = csr_matrix(etf_interaction.values)
print(etf_interaction_csr)

  (0, 414)	0.0666
  (0, 471)	0.0297
  (0, 480)	0.0338
  (0, 675)	0.0401
  (0, 862)	0.035
  (0, 1098)	0.0429
  (0, 1313)	0.0463
  (0, 1480)	0.048
  (0, 1731)	0.032
  (0, 2233)	0.0567
  (0, 2296)	0.0659
  (0, 2429)	0.0385
  (0, 2574)	0.0437
  (0, 2818)	0.0357
  (0, 3300)	0.056
  (1, 9)	0.0697
  (1, 14)	0.0169
  (1, 199)	0.0363
  (1, 215)	0.0167
  (1, 315)	0.0146
  (1, 356)	0.0147
  (1, 514)	0.0162
  (1, 1425)	0.0417
  (1, 1493)	0.0163
  (1, 1782)	0.0168
  :	:
  (1044, 1426)	0.034
  (1044, 1493)	0.0294
  (1044, 1568)	0.0195
  (1044, 1777)	0.0194
  (1044, 1782)	0.0192
  (1044, 2155)	0.055
  (1044, 2516)	0.0229
  (1044, 2530)	0.0349
  (1044, 3374)	0.0317
  (1045, 148)	0.0387
  (1045, 177)	0.0362
  (1045, 390)	0.0356
  (1045, 817)	0.0335
  (1045, 1165)	0.0332
  (1045, 1552)	0.0358
  (1045, 1690)	0.0339
  (1045, 1939)	0.0406
  (1045, 1958)	0.0328
  (1045, 2316)	0.0375
  (1045, 2436)	0.0359
  (1045, 2530)	0.0363
  (1045, 2536)	0.0375
  (1045, 2907)	0.0346
  (1045, 3343)	0.0417
  (1045, 3399)	0

#### Spliting training and test data

In [14]:
#train Test Split
x_train,x_test = random_train_test_split(etf_interaction_csr, test_percentage=0.45, random_state=90)

## Training the LightFM Model

#### Finalized Code

In [15]:
#loss1 = ("logistic","bpr","warp","logistic","bpr","warp","logistic","bpr","warp","logistic","bpr","warp")
#learning_rate1 = (0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01)
#no_components1 = (750,750,750,750,750,750,750,750,750,750,750,750,750,750,750,750,750,750,750,750)
#user_alpha1 = (0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001)
#epochs1 = (490,500,510,520,530,540,550,560,570,580,590,600,610,620,630,640,650,660)

In [16]:
#for no_components1, epochs1 in zip(no_components1, epochs1):
start = time.time()
    #print("no_components: ", no_components1, "No of Epochs:", epochs1)
model = LightFM(loss = "warp",
                random_state = 44,
                learning_rate = 0.01,
                no_components = 750,
                user_alpha = 0.01)
model = model.fit(x_train,
                  epochs = 1000,
                  num_threads=16, verbose=False)
end = time.time()
print(end - start, "Seconds")
print("Train precision: %.2f" % precision_at_k(model, x_train, k = 15).mean())
print("Test precision: %.2f" % precision_at_k(model, x_test, k = 15).mean())

255.6539077758789 Seconds
Train precision: 0.26
Test precision: 0.09


### Experiments on Hyperparameters tunning

#### Using Logistic Loss Function

In [72]:
loss1 = ("logistic","bpr","warp","logistic","bpr","warp","logistic","bpr","warp","logistic","bpr","warp","logistic","bpr","warp")
#learning_rate1 = (0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001)
#no_components1 = (100,100,100,100,100,100,100,100,100,100)
#user_alpha1 = (0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001)
epochs1 = (50,50,50,100,100,100,250,250,250,500,500,500,700,700,700)

In [67]:
for loss1,epochs1 in zip(loss1,epochs1):
    start = time.time()
    print("loss: ", loss1, "Epochs value:", epochs1)
    model = LightFM(loss = loss1,
                random_state = 44,
                learning_rate = 0.00001,
                no_components = 150,
                user_alpha = 0.00001)
    model = model.fit(x_train,
                  epochs = epochs1,
                  num_threads=16, verbose=False)
    end = time.time()
    print(end - start, "Seconds")
    print("Train precision: %.2f" % precision_at_k(model, x_train, k = 5).mean())
    print("Test precision: %.2f" % precision_at_k(model, x_test, k = 5).mean())

loss:  logistic Epochs value: 50
61.8465518951416 Seconds
Train precision: 0.12
Test precision: 0.09
loss:  bpr Epochs value: 50
36.90430402755737 Seconds
Train precision: 0.06
Test precision: 0.03
loss:  warp Epochs value: 50
36.35699105262756 Seconds
Train precision: 0.12
Test precision: 0.09
loss:  logistic Epochs value: 100
48.91942381858826 Seconds
Train precision: 0.12
Test precision: 0.09
loss:  bpr Epochs value: 100
71.80420541763306 Seconds
Train precision: 0.08
Test precision: 0.05
loss:  warp Epochs value: 100
73.63605237007141 Seconds
Train precision: 0.13
Test precision: 0.08
loss:  logistic Epochs value: 250
126.02750277519226 Seconds
Train precision: 0.12
Test precision: 0.09
loss:  bpr Epochs value: 250
182.8010971546173 Seconds
Train precision: 0.09
Test precision: 0.06
loss:  warp Epochs value: 250
182.58670926094055 Seconds
Train precision: 0.13
Test precision: 0.08
loss:  logistic Epochs value: 500
243.94394731521606 Seconds
Train precision: 0.12
Test precision: 0.0

#### Using Warp Loss Function

In [14]:
#loss1 = ("logistic","bpr","warp")
learning_rate1 = (0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0)
no_components1 = (100,200,300,400,500,600,700,800,900,1000)
user_alpha1 = (0.00000001,0.00000002,0.00000003,0.00000004,0.00000005,0.00000006,0.00000007,0.00000008,0.00000009,0.00000010)
k1 = (5,5,5,5,5,10,10,10,10,10)

In [15]:
for learning_rate1, no_components1, user_alpha1, k1 in zip(learning_rate1, no_components1, user_alpha1, k1):
    start = time.time()
    print("Learning rate: ", learning_rate1, "No. of components:", no_components1, "user_alpha Value:", user_alpha1, "Precision K Value:", k1)
    model = LightFM(loss = "warp",
                random_state = 44,
                learning_rate = learning_rate1,
                no_components = no_components1,
                user_alpha = user_alpha1)
    model = model.fit(x_train,
                  epochs=500,
                  num_threads=16, verbose=False)
    end = time.time()
    print(end - start, "Seconds")
    print("Train precision: %.2f" % precision_at_k(model, x_train, k = k1).mean())
    print("Test precision: %.2f" % precision_at_k(model, x_test, k = k1).mean())

Learning rate:  0.1 No. of components: 100 user_alpha Value: 1e-08 Precision K Value: 5
29.084501028060913 Seconds
Train precision: 1.00
Test precision: 0.00
Learning rate:  0.2 No. of components: 200 user_alpha Value: 2e-08 Precision K Value: 5
67.14807367324829 Seconds
Train precision: 0.80
Test precision: 0.02
Learning rate:  0.3 No. of components: 300 user_alpha Value: 3e-08 Precision K Value: 5
664.5807592868805 Seconds
Train precision: 0.01
Test precision: 0.00
Learning rate:  0.4 No. of components: 400 user_alpha Value: 4e-08 Precision K Value: 5
931.7359962463379 Seconds
Train precision: 0.01
Test precision: 0.00
Learning rate:  0.5 No. of components: 500 user_alpha Value: 5e-08 Precision K Value: 5
1163.214810848236 Seconds
Train precision: 0.01
Test precision: 0.00
Learning rate:  0.6 No. of components: 600 user_alpha Value: 6e-08 Precision K Value: 10
1392.0732898712158 Seconds
Train precision: 0.01
Test precision: 0.00
Learning rate:  0.7 No. of components: 700 user_alpha V

#### Using brp Loss Function

In [22]:
#loss1 = ("logistic","bpr","warp")
learning_rate1 = (0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0)
no_components1 = (100,200,300,400,500,600,700,800,900,1000)
user_alpha1 = (0.00000001,0.00000002,0.00000003,0.00000004,0.00000005,0.00000006,0.00000007,0.00000008,0.00000009,0.00000010)
k1 = (5,5,5,5,5,10,10,10,10,10)

In [23]:
for learning_rate1, no_components1, user_alpha1, k1 in zip(learning_rate1, no_components1, user_alpha1, k1):
    start = time.time()
    print("Learning rate: ", learning_rate1, "No. of components:", no_components1, "user_alpha Value:", user_alpha1, "Precision K Value:", k1)
    model = LightFM(loss = "bpr",
                random_state = 44,
                learning_rate = learning_rate1,
                no_components = no_components1,
                user_alpha = user_alpha1)
    model = model.fit(x_train,
                  epochs=500,
                  num_threads=16, verbose=False)
    end = time.time()
    print(end - start, "Seconds")
    print("Train precision: %.2f" % precision_at_k(model, x_train, k = k1).mean())
    print("Test precision: %.2f" % precision_at_k(model, x_test, k = k1).mean())

Learning rate:  0.1 No. of components: 100 user_alpha Value: 1e-08 Precision K Value: 5
256.8566598892212 Seconds
Train precision: 0.99
Test precision: 0.00
Learning rate:  0.2 No. of components: 200 user_alpha Value: 2e-08 Precision K Value: 5
493.0964922904968 Seconds
Train precision: 1.00
Test precision: 0.00
Learning rate:  0.3 No. of components: 300 user_alpha Value: 3e-08 Precision K Value: 5
701.282488822937 Seconds
Train precision: 1.00
Test precision: 0.00
Learning rate:  0.4 No. of components: 400 user_alpha Value: 4e-08 Precision K Value: 5
765.6007242202759 Seconds
Train precision: 1.00
Test precision: 0.00
Learning rate:  0.5 No. of components: 500 user_alpha Value: 5e-08 Precision K Value: 5
795.6251351833344 Seconds
Train precision: 0.99
Test precision: 0.00
Learning rate:  0.6 No. of components: 600 user_alpha Value: 6e-08 Precision K Value: 10
930.1951313018799 Seconds
Train precision: 0.94
Test precision: 0.01
Learning rate:  0.7 No. of components: 700 user_alpha Valu

## Definitions of the recomendation Model

In [17]:
def sample_recommendation_user(model, etf_df, ETFSym, user_dict, 
                               item_dict,threshold = 0,nrec_items = 15, show = True):
    
    n_users, n_items = etf_df.shape
    #print(n_users, n_items)
    user_x = user_dict[ETFSym]
    #print(user_x)
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    
    
    scores.index = etf_df.columns
    #print(scores)
    # scores_dict{}
    pred_scores=scores.to_dict()
#     print(pred_scores)
    pred_stocks = pd.Series(scores.sort_values(ascending=False).index)
    #print(scores)
    
#     known_items = list(pd.Series(Books_df1.loc[ETFSym,:] \
#     [etf_df.loc[ETFSym,:] > threshold].index).sort_values(ascending=False))
    
   # print(known_items)
    
   # scores = [x for x in pred_stocks if x not in known_items]
   # print(scores)
    return_stock_list = pred_stocks[0:nrec_items]
    #print(return_score_list)
#     print(known_items)
    #print(item_dict)
#     known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    pred_percentage = pd.Series(return_stock_list).apply(lambda x: pred_scores[x])
    if show == True:
        print ("User: " + str(ETFSym))
    
       # print("Known Likes:")
       # counter= 1
        
        #for i in known_items:
         #   print(i)
          #  print(str(counter) + '- ' +str(i))
           # counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in return_stock_list:
            print(str(counter) + '- ' + str(i) + str(pred_scores[i]))
            counter+=1

## Testing the model for an ETF & displaying the recomendations

In [21]:
print(sample_recommendation_user(model, etf_interaction, 'ACVF', user_dict, itemdict))

HoldingSym
A       0.066931
AA      0.118252
AAAU   -0.039013
AAL     0.072339
AALB   -0.316049
          ...   
ZUMZ   -0.013317
ZUO    -0.125915
ZURN    0.009315
ZWS    -0.153425
ZYNE   -0.154076
Length: 3681, dtype: float32
User: ACVF

 Recommended Items:
1- AAPL1.9195820093154907
2- MSFT1.9179942607879639
3- GOOGL1.823801040649414
4- PG1.7725952863693237
5- AMZN1.7609198093414307
6- V1.7369053363800049
7- UNH1.7360389232635498
8- JNJ1.734400749206543
9- NVDA1.7077311277389526
10- HD1.7075566053390503
11- TSLA1.7020561695098877
12- FB1.6684449911117554
13- JPM1.6643093824386597
14- BRK.B1.6614149808883667
15- GOOG1.6338930130004883
None


In [20]:
print(sample_recommendation_user(model, etf_interaction, 'DVOL', user_dict, itemdict))

HoldingSym
A       0.562365
AA      0.216851
AAAU   -0.298670
AAL     0.221832
AALB   -0.363110
          ...   
ZUMZ   -0.200494
ZUO    -0.253159
ZURN   -0.206874
ZWS    -0.267664
ZYNE   -0.435246
Length: 3681, dtype: float32
User: DVOL

 Recommended Items:
1- ABBV1.6597756147384644
2- LMT1.6060562133789062
3- AAPL1.60358726978302
4- VZ1.5976613759994507
5- XOM1.5818973779678345
6- AVGO1.5620888471603394
7- PEP1.5609544515609741
8- PLD1.5582895278930664
9- CVX1.5295050144195557
10- AMGN1.5264335870742798
11- WMT1.5172905921936035
12- PSA1.5000908374786377
13- COST1.4927446842193604
14- BMY1.4789544343948364
15- JNJ1.4765686988830566
None


## Final accuracy measures

In [22]:
auc_score(model, x_test, x_train, user_features=None, item_features=None, preserve_rows=False, num_threads=1, check_intersections=True)

array([0.31878525, 0.99448997, 0.97821736, ..., 0.6120815 , 0.99945444,
       0.9440655 ], dtype=float32)

In [23]:
print("Train auc score: %.2f" % auc_score(model, x_train).mean())
print("Test auc score: %.2f" % auc_score(model, x_test).mean())

Train auc score: 0.96
Test auc score: 0.89


In [24]:
print("Train precision: %.2f" % precision_at_k(model, x_train, k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, x_test, k=5).mean())

Train precision: 0.33
Test precision: 0.10
